In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
df = pd.read_excel(r'DVDK_DL_BB_1.xlsx')
loai = pd.read_json('loai.json')

#print(df)
loai.head(6)


,ID,Ten,TenKhoaHoc,ChiID
0,532,Cá bò ba gai mõm ngắn,"Triacanthus biaculeatus (Bloch, 1786)",158
1,531,Cá bơn lưỡi,"Cynoglosssus monopus (Bleeker, 1849)",135
2,530,Cá bống râu,"Triaenopogon barbatus (Gunther, 1861)",183
3,529,Cá kẽm hoa,"Plectorhynchus pictus (Thunberg, 1792)",149
4,528,Cá hồng vàng sọc mờ,"Lutianus lineolatus (Ruppell, 1829)",148
5,527,Cá hồng chấm,"Lutianus johnii (Bloch, 1792)",148


In [4]:
df.info()
print('Shape:', df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   STT             12 non-null     object
 1   Tên khoa học    39 non-null     object
 2   Tên Tiếng Việt  5 non-null      object
dtypes: object(3)
memory usage: 1.0+ KB
Shape: (39, 3)


In [5]:
professor = df[df.STT.notnull()]
professor.head(5)
professor['Tên khoa học'].values

array(['Lumbriconereis notocirrata Fauvel, 1932\xa0\xa0',
       'Marphysa mossambica Peters, 1854',
       'Ancistrosyllis constricta  (Southern, 1921)',
       'Nephthys californiensis Hartman, 1938',
       'Nephthys gravieriAugener, 1913',
       'Ceratonereis burmensis Monro, 1937',
       'Dendronereis arborifera  Peter, 1854',
       'Tylonereis bogoyawlenskyi Fauvel, 1911',
       'Scolelepis indica Fauvel 1928', 'Prionospio pinnata Ehlers, 1901',
       'Holothuria scabra Jaeger, 1833',
       'Colochirus quandranularis Troschel, 1846'], dtype=object)

In [25]:
for element in professor['Tên khoa học'].values:
    #print(element.strip())
    check = False
    tmp = element.strip()
    for loai_element in loai.values:
        if element.strip()== loai_element[2].strip():
            check = True
            tmp = loai_element[2].strip() + "#" + str(loai_element[0])
    if check==False:
        tmp = element.strip() + "#0"
    print(tmp)
        

Lumbriconereis notocirrata Fauvel, 1932#357
Marphysa mossambica Peters, 1854#0
Ancistrosyllis constricta  (Southern, 1921)#359
Nephthys californiensis Hartman, 1938#360
Nephthys gravieriAugener, 1913#0
Ceratonereis burmensis Monro, 1937#362
Dendronereis arborifera  Peter, 1854#364
Tylonereis bogoyawlenskyi Fauvel, 1911#366
Scolelepis indica Fauvel 1928#0
Prionospio pinnata Ehlers, 1901#0
Holothuria scabra Jaeger, 1833#0
Colochirus quandranularis Troschel, 1846#369


In [28]:
df_demo = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('BIỂU MẪU - BIỂU PHÂN TÍCH MẪU CÁ#2065.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_demo.to_excel(writer, sheet_name='LoaiDaDangSinhHoc', startrow=3)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [29]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
        
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [31]:
df_demo = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})
append_df_to_excel("BIỂU MẪU - BIỂU PHÂN TÍCH MẪU CÁ#2065.xlsx",df_demo,"LoaiDaDangSinhHoc", 3)